<a href="https://colab.research.google.com/github/helomelo1/Implementation-of-Some-Simple-Trading-Strats/blob/main/makemore_using_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
words = open('names.txt', 'r').read().splitlines()
print(words[:5])

['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [78]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# print(itos)

In [79]:
block_size = 3
X, y = [], []

for w in words:
    # print('\n', w)
    context = [0] * block_size

    for ch in w + '.':
        idx = stoi[ch]
        X.append(context)
        y.append(idx)

        # print(''.join(itos[i] for i in context), '->', itos[idx])

        context = context[1:] + [idx]

X = torch.tensor(X)
y = torch.tensor(y)

In [80]:
print(X.shape)
print(y.shape)

torch.Size([228146, 3])
torch.Size([228146])


In [81]:
c = torch.randn((27, 2))
embedding = c[X]
print(embedding.shape)

torch.Size([228146, 3, 2])


In [82]:
w1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [83]:
h = torch.matmul(embedding.view(-1, 6), w1) + b1
h = torch.tanh(h)

In [84]:
w2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [85]:
logits = torch.matmul(h, w2) + b2
logits.shape

torch.Size([228146, 27])

In [86]:
counts = logits.exp()

In [87]:
probs = counts / counts.sum(1, keepdim=True)
probs.shape

torch.Size([228146, 27])

In [88]:
# REARRANGING STUFF BELOW

In [100]:
g = torch.Generator().manual_seed(0)
c = torch.randn((27, 2), generator=g)

w1 = torch.randn((6, 100))
b1 = torch.randn(100)

w2 = torch.randn((100, 27))
b2 = torch.randn(27)

parameters = [c, w1, b1, w2, b2]

In [101]:
sum(p.nelement() for p in parameters)

3481

In [102]:
for p in parameters:
    p.requires_grad = True

In [103]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [118]:
lri = []
lossi = []

for i in range(10000):
    idx = torch.randint(0, X.shape[0], (32, ))

    emb = c[X[idx]]

    h = torch.matmul(emb.view(-1, 6), w1) + b1
    h = torch.tanh(h)

    logits = torch.matmul(h, w2) + b2
    loss = F.cross_entropy(logits, y[idx])

    for p in parameters:
        p.grad = None
    loss.backward()

    # lr = lrs[i]
    for p in parameters:
        p.data += -0.1 * p.grad

    # lri.append(lrs[i])
    # lossi.append(loss.item())

    # if i % 100 == 0:
    #     print(f'Epoch {i}: {loss.item()}')

print(loss.item())

1.9204038381576538


In [119]:
emb = c[X]
h = torch.matmul(emb.view(-1, 6), w1) + b1
h = torch.tanh(h)
logits = torch.matmul(h, w2) + b2
loss = F.cross_entropy(logits, y)
loss.item()

2.3281571865081787